In [2]:
import pandas as pd

In [4]:
airline_df = pd.read_csv('data\\raw\\airlines.csv')
airline_df.sample(5)

,IATA_CODE,AIRLINE
5,OO,Skywest Airlines Inc.
12,MQ,American Eagle Airlines Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
13,VX,Virgin America


In [5]:
airport_df = pd.read_csv('data\\raw\\airports.csv')
airport_df.sample(5)

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
316,VPS,Destin-Fort Walton Beach Airport/Eglin AFB,Valparaiso,FL,USA,30.48325,-86.52540
22,AUS,Austin-Bergstrom International Airport,Austin,TX,USA,30.19453,-97.66987
159,ISN,Sloulin Field International Airport,Williston,ND,USA,48.17794,-103.64235
105,EWN,Coastal Carolina Regional Airport (Craven Coun...,New Bern,NC,USA,35.07297,-77.04294
117,FSM,Fort Smith Regional Airport,Fort Smith,AR,USA,35.33659,-94.36744


In [6]:
flight_df = pd.read_csv('data\\raw\\flights.csv')
flight_df.sample(5)

C:\Users\user\AppData\Local\Temp\ipykernel_14184\2406861582.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flight_df = pd.read_csv('data\\raw\\flights.csv')


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
5632350,2015,12,19,6,AA,2430,N4XLAA,CMH,DFW,1910,...,2037.0,-31.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4156359,2015,9,16,3,DL,1151,N372DA,MSP,SFO,1140,...,1349.0,7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1180492,2015,3,18,3,DL,1967,N953AT,CHA,ATL,1105,...,1146.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2919188,2015,7,2,4,WN,4295,N496WN,HOU,EWR,1725,...,2146.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2229914,2015,5,22,5,OO,4596,N824SK,GTF,MSP,600,...,907.0,-11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
cancel_flight_df = flight_df[['CANCELLED', 'CANCELLATION_REASON']]
flight_df.drop(columns=['CANCELLED', 'CANCELLATION_REASON'], inplace=True)
cancel_flight_df.sample(5)

,CANCELLED,CANCELLATION_REASON
1813609,0,NaN
4224539,0,NaN
202199,0,NaN
3168785,0,NaN
5461290,0,NaN


In [12]:
flight_df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
       'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')